In [7]:
import pandas as pd
import glob

def merge_csv_files(input_files, output_file_name):

    # Merge multiple CSV files in a folder into a single CSV file

    df_list = []

    for filename in input_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)

    combined_df = pd.concat(df_list, axis=0, ignore_index=True)
    combined_df.to_csv(output_file_name, index=False)
   

merge_csv_files(["2021VAERSSYMPTOMS.csv","2022VAERSSYMPTOMS.csv","2023VAERSSYMPTOMS.csv"], "VAERSSYMPTOMS.csv")

(989639, 11)
(330825, 11)
(26439, 11)
(1346903, 11)


(1346903, 11)
